In [ ]:
# making things autorelad - needed for Jupyter Kernel
%load_ext autoreload
%autoreload 2
%matplotlib widget

# define the right-path
import sys
import os
# TODO: remove this hardcoding
folder_path = '/home/jellepoland/surfdrive/phd/code/phd_Jelle_Poland/Simulations'
os.chdir(folder_path)  # This should not be needed
sys.path.append(os.getcwd())

# Time to import the modules
from src.initialisation import load_surfplan, pulley_connectivity
from src.initialisation.input_classes import input_VSM , input_bridle_aero, input_structural_solver
from src.initialisation import input_particleSystem,particles_with_rotational_resistance
from src.particleSystem.ParticleSystem import ParticleSystem
from src.coupling import coupling_aero2struc, coupling_struc2aero
from src.structural import structural_model, structural_mesher
from src.solver import solver_main, solver_utils
from src.post_processing import functions_print, functions_plot, post_processing_utils, post_processing_main
from src import actuation_relations
from src.aerodynamic import VSM, breukels_2D, plate_aero, bridle_line_system_aero, tether_aero

from test import test_main
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import yaml
import importlib
import pytest
import pandas as pd
import dill

# {} gives units. {{}} is normal {} in Latexs
from IPython.display import display, Latex

## All Immutatbles are stored in the dataclass config, i.e. simulation settings and configuration of the kite 
from src.initialisation.yaml_loader import config


##TODO: what has to be adjusted to preference
folder_name = f"results/{config.kite_name}/torque_paper/2023_26_13_v3/"
is_with_printing = True
is_with_plotting = True
is_with_animation = False

data_names = [
    "points",
    "psystem",
    "print_data",
    "plot_data",
    "animation_data",
    "config",
    "vel_app",
    "sim_name",
    "input_VSM",
]
loaded_data = {}
for name in data_names:
    with open(f"{folder_name}{name}.pkl", "rb") as f:
        loaded_data[name] = dill.load(f)

points = loaded_data["points"]
psystem = loaded_data["psystem"]
print_data = loaded_data["print_data"]
plot_data = loaded_data["plot_data"]
animation_data = loaded_data["animation_data"]
config = loaded_data["config"]
vel_app = loaded_data["vel_app"]
sim_name = loaded_data["sim_name"]
input_VSM = loaded_data["input_VSM"]

if is_with_printing:
    post_processing_main.print_results(
        points,
        print_data,
        config,
    )
if is_with_plotting:
    post_processing_main.plot(
        plot_data,
        points,
        vel_app,
        config,
    )
if is_with_animation:
    post_processing_main.animate(
        animation_data,
        vel_app,
        sim_name,
        config,
        input_VSM,
    )


In [ ]:
r_0 = 28
force_aero = print_data[8][0]
mass_points = config.kite.mass_points
length_tether = 200
is_print_intermediate_results = False

(
    force_centrifugal,
    r_0,
    v_k_i_array,
    center_of_gravity,
) = solver_utils.calculate_force_centrifugal_distribution_with_tether_tension(
    r_0,
    force_aero,
    mass_points,
    vel_app,
    points,
    length_tether,
    is_print_intermediate_results,
)
print(f"force_centrifugal = {np.sum(force_centrifugal[:,1])}")